In [1]:
import subprocess
import json
from dotenv import load_dotenv
import os

from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI

from web3 import Web3

from eth_account import Account

In [2]:
load_dotenv()
from constants import *
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:7545"))

In [3]:
mnemonic = os.getenv('MNEMONIC')
print(mnemonic)

raccoon pattern volume heart idea beyond ivory frozen vanish spirit blood ancient club short parade


In [4]:
def derive_wallets(phrase, coin, numderive):
    command = f'php derive -g --mnemonic="{phrase}" --numderive={numderive} --coin={coin} --format=jsonpretty'
    p = subprocess.Popen(command,stdout=subprocess.PIPE,shell=True)
    (output, err) = p.communicate()
    return json.loads(output)

In [5]:
 coins = {'eth':derive_wallets(phrase=mnemonic,coin=ETH,numderive=3),'btc-test': derive_wallets(phrase=mnemonic,coin=BTCTEST,numderive=3)}
print(json.dumps(coins, indent=4, sort_keys=True))

{
    "btc-test": [
        {
            "address": "mu22pWKxQT7xPqqw2iiKNeqonHtDCDPajS",
            "index": 0,
            "path": "m/44'/1'/0'/0/0",
            "privkey": "cP7m7idh46aPQUoPCKjbAMgd2TUNQb7Vwr6yB3fVQRzi8fpWoeT5",
            "pubkey": "03772db5cf5d6c21fb34ba74f3aa2cc06c4b4b8edddcb162191f819e05df82407e",
            "pubkeyhash": "941979b38940a144911e95aa7704ef6994ed5134",
            "xprv": "tprv8jwx6Wnwhd5WxntpdWo6EtXSD6aVRMGaeZWuPG6Bpz2NoAn98pyymFa9xC2WiNvT33wAci985P6GcZFwfuQDppgTFs2UnfWuo4ecsNth3wp",
            "xpub": "tpubDGdzEvqBqzmBrFvcXATgeJBYn86RagTVDs7gfn8VFFpmdf2umDoZwkC28MUTomEYU2wR3G1n38uMEvzZNVbyfSp4tNX9pwo6sSDUsq6Lscb"
        },
        {
            "address": "mrKCmUZpJFjmeUwjNwniwPNaotaVWvGPdz",
            "index": 1,
            "path": "m/44'/1'/0'/0/1",
            "privkey": "cNpzohfKkGU64zuZmUufRCnRdtGVBifF7NRTG9ce5yZFEWvuPwQk",
            "pubkey": "020d31a11a55e55151183035e25e2848a11052c8413a110a57122ecc6128dbe47d",
            "pubkeyh

In [6]:
eth_key = coins['eth'][1]['privkey']
btc_key = coins['btc-test'][0]['privkey']

In [7]:
def priv_key_to_account(coin, priv_key):
    global account
    if coin == 'eth':
        account=Account.privateKeyToAccount(priv_key)
    else:
        account=PrivateKeyTestnet(priv_key)
    return account

In [8]:
priv_key_to_account(ETH,eth_key)

In [9]:
priv_key_to_account(BTCTEST,btc_key)

<PrivateKeyTestnet: mu22pWKxQT7xPqqw2iiKNeqonHtDCDPajS>

In [10]:
def create_tx(coin,account,recipient,amount):
    global tx_data
    if coin =='eth':
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        tx_data= {
            "from": account.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
        }
        return tx_data
    else:
        tx_data = PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])
        return tx_data

In [25]:
create_tx(BTCTEST,account,'mrKCmUZpJFjmeUwjNwniwPNaotaVWvGPdz',0.0001)

In [ ]:
create_tx(ETH,account,'0x21E3C13737174662EA28e850484e0d72819a3D45',0.0001)

In [ ]:
def send_tx(coin, account, recipient, amount):
    if coin == ETH:
        tx = create_tx(coin,account, recipient, amount)
        signed_tx = account.sign_transaction(tx)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        tx_data = create_tx(coin,account,recipient,amount)
        signed = account.sign_transaction(tx_data)
        NetworkAPI.broadcast_tx_testnet(signed)
        return signed

In [ ]:
send_tx(BTCTEST,account,'mu22pWKxQT7xPqqw2iiKNeqonHtDCDPajS',0.0001)